In [1]:
# Import libaries
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Step 1: Create a soup object from the home page

In [5]:
url = 'http://chubbygrub.com/'
res = requests.get(url)
res.status_code

200

In [6]:
soup = BeautifulSoup(res.content, 'lxml')

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and slug. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}, 
    {'name': "Applebee's", 'slug': 'applebees'},
    ...
]
```

In [18]:
a = soup.find_all('a', {'class' : 'btn btn-lg btn-primary'})
names = [ai.text for ai in a]
# names

In [31]:
slugs = [ai.attrs['href'].replace('/restaurants/','') for ai in a]

In [39]:
restaurants = []
for k, v in zip(names, slugs):
    dictionary = {}
    dictionary['name'] = k
    dictionary['slugs'] = v
    restaurants.append(dictionary)
    
# restaurants

### Step 3: Using the slug, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [44]:
base_url = 'http://chubbygrub.com'

In [46]:
urls_list = [base_url + ai.attrs['href'] for ai in a]

In [49]:
# testing out for one page: 
url = 'http://chubbygrub.com/restaurants/kfc/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')

In [53]:
table = soup.find('table', {'id' : 'items'})

In [92]:
food = []
for row in table.find_all('tr')[1:]:
#     print(row)
    items = {}
    
    items['calories'] = row.find('td', {'itemprop' : 'calories'}).text
    items['carbs'] = row.find('td', {'itemprop' : 'carbohydrateContent'}).text
    items['category'] = row.find('td', {'class' : 'hidden-xs category-column'}).text.strip()
    items['fat'] = row.find('td', {'itemprop' : 'fatContent'}).text
    items['name'] = row.find('td', {'itemprop' : 'name'}).text
    items['restaurant'] = soup.find('span', {'itemprop' : 'name'}).text
    
    food.append(items)
food 

[{'calories': '250',
  'carbs': '33',
  'category': 'Desserts',
  'fat': '12',
  'name': 'Apple Turnover (1)',
  'restaurant': 'KFC'},
 {'calories': '210',
  'carbs': '41',
  'category': 'Sides',
  'fat': '1.5',
  'name': 'BBQ Baked Beans',
  'restaurant': 'KFC'},
 {'calories': '180',
  'carbs': '23',
  'category': 'Sides',
  'fat': '8',
  'name': 'Biscuit',
  'restaurant': 'KFC'},
 {'calories': '40',
  'carbs': '2',
  'category': 'Salads',
  'fat': '2',
  'name': 'Caesar Side Salad without Dressing & Croutons',
  'restaurant': 'KFC'},
 {'calories': '300',
  'carbs': '39',
  'category': 'Desserts',
  'fat': '15',
  'name': 'Café Valley Bakery® Chocolate Chip Cake (6 slices per cake)',
  'restaurant': 'KFC'},
 {'calories': '790',
  'carbs': '66',
  'category': 'Chicken',
  'fat': '45',
  'name': 'Chicken Pot Pie',
  'restaurant': 'KFC'},
 {'calories': '150',
  'carbs': '21',
  'category': 'Sides',
  'fat': '6',
  'name': 'Cole Slaw',
  'restaurant': 'KFC'},
 {'calories': '70',
  'carbs'

In [113]:
def extract_stuff(soup, food_list = food):
    for row in table.find_all('tr')[1:]:
        items = {}
        items['calories'] = row.find('td', {'itemprop' : 'calories'}).text
        items['carbs'] = row.find('td', {'itemprop' : 'carbohydrateContent'}).text
        items['category'] = row.find('td', {'class' : 'hidden-xs category-column'}).text.strip()
        items['fat'] = row.find('td', {'itemprop' : 'fatContent'}).text
        items['name'] = row.find('td', {'itemprop' : 'name'}).text
        items['restaurant'] = soup.find('span', {'itemprop' : 'name'}).text
    
        food_list.append(items)
    return food_list

In [114]:
food = []
soup = BeautifulSoup(res.content, 'lxml')
extract_stuff(soup)

[{'calories': '450',
  'carbs': '24',
  'category': 'Chicken',
  'fat': '30',
  'name': '10-piece Chicken Nuggets',
  'restaurant': "Wendy's"},
 {'calories': '430',
  'carbs': '23',
  'category': 'Chicken',
  'fat': '28',
  'name': '10-piece Spicy Chicken Nuggets',
  'restaurant': "Wendy's"},
 {'calories': '180',
  'carbs': '10',
  'category': 'Chicken',
  'fat': '12',
  'name': '4-piece Chicken Nuggets',
  'restaurant': "Wendy's"},
 {'calories': '170',
  'carbs': '9',
  'category': 'Chicken',
  'fat': '11',
  'name': '4-piece Spicy Chicken Nuggets',
  'restaurant': "Wendy's"},
 {'calories': '270',
  'carbs': '14',
  'category': 'Chicken',
  'fat': '18',
  'name': '6-piece Chicken Nuggets',
  'restaurant': "Wendy's"},
 {'calories': '260',
  'carbs': '14',
  'category': 'Chicken',
  'fat': '17',
  'name': '6-piece Spicy Chicken Nuggets',
  'restaurant': "Wendy's"},
 {'calories': '570',
  'carbs': '49',
  'category': 'Salads',
  'fat': '27',
  'name': 'Apple Pecan Chicken Salad',
  'rest

In [95]:
from time import sleep #or import time, then later time.sleep(seconds)

In [93]:
food = []
for u in urls_list:
    res = requests.get(u)
    sleep(1)
    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find('table', {'id' : 'items'})
    
    for row in table.find_all('tr')[1:]:
        items = {}    
        items['calories'] = row.find('td', {'itemprop' : 'calories'}).text
        items['carbs'] = row.find('td', {'itemprop' : 'carbohydrateContent'}).text
        items['category'] = row.find('td', {'class' : 'hidden-xs category-column'}).text.strip()
        items['fat'] = row.find('td', {'itemprop' : 'fatContent'}).text
        items['name'] = row.find('td', {'itemprop' : 'name'}).text
        items['restaurant'] = soup.find('span', {'itemprop' : 'name'}).text
        
        food.append(items)

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 4,977 rows

In [97]:
df = pd.DataFrame(food)

In [98]:
df.shape

(4977, 6)

### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [99]:
%pwd

'/Users/hayatoumy/Documents/GA files/git_lessons/5.01-lab-beautiful_soup-webscraping'

In [100]:
df.to_csv('food_items.csv', index = False) #if index is true, adds another id column!